In [6]:
from ultralytics import YOLO

model = YOLO("yolo11x-pose.pt")

model.to('cuda')

import os 
# loop through all the files in the folder
results = []

#for filename in os.listdir('images'):
for filename in os.listdir('validation_dataset'):
    print(filename)
    #results = model.predict("images/"+filename, imgsz=320, conf=0.5) + results
    results = model.predict("validation_dataset/"+filename, imgsz=320, conf=0.5) + results


#results = model.predict("images/hammer curl",save= True, imgsz=320, conf=0.5, device = 'gpu')





bench press

image 1/839 c:\Users\Samet\Desktop\3. Grade 1. Term\AIN 311 - 313\project\yolo model\validation_dataset\bench press\0.00000258.jpg: 320x192 1 person, 73.8ms
image 2/839 c:\Users\Samet\Desktop\3. Grade 1. Term\AIN 311 - 313\project\yolo model\validation_dataset\bench press\0.00226286.jpg: 320x192 (no detections), 64.0ms
image 3/839 c:\Users\Samet\Desktop\3. Grade 1. Term\AIN 311 - 313\project\yolo model\validation_dataset\bench press\0.00241480.jpg: 320x192 (no detections), 28.0ms
image 4/839 c:\Users\Samet\Desktop\3. Grade 1. Term\AIN 311 - 313\project\yolo model\validation_dataset\bench press\0.00377820.jpg: 320x192 1 person, 30.0ms
image 5/839 c:\Users\Samet\Desktop\3. Grade 1. Term\AIN 311 - 313\project\yolo model\validation_dataset\bench press\0.00454068.jpg: 320x192 1 person, 37.0ms
image 6/839 c:\Users\Samet\Desktop\3. Grade 1. Term\AIN 311 - 313\project\yolo model\validation_dataset\bench press\0.00630258.jpg: 320x192 1 person, 27.0ms
image 7/839 c:\Users\Samet\Desk

In [7]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))  # Optional: Display GPU name

True
NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [8]:
import torch
import math
import csv

# Joint labels for reference
joint_labels = ["Nose", "Left Eye", "Right Eye", "Left Ear", "Right Ear",
                "Left Shoulder", "Right Shoulder", "Left Elbow", "Right Elbow",
                "Left Wrist", "Right Wrist", "Left Hip", "Right Hip",
                "Left Knee", "Right Knee", "Left Ankle", "Right Ankle"]

# Function to calculate angle between three points (p1, p2, p3)
def calculate_angle(p1, p2, p3):
    # Create vectors
    v1 = p1 - p2
    v2 = p3 - p2
    
    # Calculate dot product and magnitudes
    dot_product = torch.dot(v1, v2)
    magnitude_v1 = torch.norm(v1)
    magnitude_v2 = torch.norm(v2)
    
    # Prevent division by zero
    if magnitude_v1 == 0 or magnitude_v2 == 0:
        return None  # Undefined angle
    
    # Calculate angle in radians
    angle_rad = torch.acos(dot_product / (magnitude_v1 * magnitude_v2))
    # Convert to degrees
    return math.degrees(angle_rad.item())

# Function to calculate Euclidean distance between two points (p1, p2)
def calculate_distance(p1, p2):
    return torch.norm(p1 - p2).item()

# List to store calculated angles and distances for each image
angles_per_image = []

# Define joint triplets for angle calculations
joint_triplets = [
    (5, 7, 9),  # Left Elbow
    (6, 8, 10), # Right Elbow
    (5, 11, 13),# Left Hip
    (6, 12, 14),# Right Hip
    (7, 5, 11), # Left Shoulder
    (8, 6, 12), # Right Shoulder
    (11, 13, 15),# Left Knee
    (12, 14, 16) # Right Knee
]

# Iterate over each image in the batch
for img_idx in range(len(results)):  # Iterate over images (batch size)
    if len(results[img_idx]) == 0:
        print(f"No keypoints detected in image {img_idx + 1}")
        continue
    if len(results[img_idx]) > 1:
        print(f"Multiple detections in image {img_idx + 1}")
        continue
    image_angles = {"Image": img_idx + 1}
    
    # Calculate angles and distances for each joint triplet
    for triplet in joint_triplets:
        p1 = results[img_idx].keypoints.xy[0][triplet[0]]  # Correctly access the joint coordinates
        p2 = results[img_idx].keypoints.xy[0][triplet[1]]
        p3 = results[img_idx].keypoints.xy[0][triplet[2]]
        
        # Make sure the coordinates are tensors and have the shape [2] (x, y)
        angle = calculate_angle(p1, p2, p3)
        image_angles[f"{joint_labels[triplet[0]]}-{joint_labels[triplet[1]]}-{joint_labels[triplet[2]]}"] = angle
    
    # Calculate and add distances for each joint pair
    for i in range(len(results[img_idx].keypoints.xy[0])):
        for j in range(i + 1, len(results[img_idx].keypoints.xy[0])):
            p1 = results[img_idx].keypoints.xy[0][i]
            p2 = results[img_idx].keypoints.xy[0][j]
            distance = calculate_distance(p1, p2)
            image_angles[f"{joint_labels[i]}-{joint_labels[j]}_distance"] = distance

    #add the positions of the joints
    for i in range(len(results[img_idx].keypoints.xy[0])):
        p1 = results[img_idx].keypoints.xy[0][i]
        y_pos = p1[1]
        # convert tensor to float
        y_pos = y_pos.item()

        x_pos = p1[0]
        # convert tensor to float
        x_pos = x_pos.item()

        image_angles[f"{joint_labels[i]}_position_y"] = y_pos
        image_angles[f"{joint_labels[i]}_position_x"] = x_pos
        

        # to detect the position of the joint in the image

    

    
    
    # Add the "Label" field to each image
    image_angles["Label"] = results[img_idx].path.split("\\")[-2]
    
    angles_per_image.append(image_angles)

# Save the angles and distances for all images as a CSV file
#
# csv_file_path = "joint_angles_and_distances_per_image.csv"
csv_file_path = "validation_dataset.csv"
with open(csv_file_path, mode="w", newline="") as csv_file:
    # Get fieldnames (the keys of the first entry in angles_per_image)
    fieldnames = angles_per_image[0].keys()
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    # Write header and rows
    writer.writeheader()
    writer.writerows(angles_per_image)

print(f"Angles and distances for each image saved to {csv_file_path}")


Multiple detections in image 27
Multiple detections in image 29
No keypoints detected in image 34
Multiple detections in image 122
No keypoints detected in image 235
Multiple detections in image 277
No keypoints detected in image 320
Multiple detections in image 339
No keypoints detected in image 505
No keypoints detected in image 559
Multiple detections in image 629
Multiple detections in image 681
Multiple detections in image 741
No keypoints detected in image 802
Multiple detections in image 837
No keypoints detected in image 890
Multiple detections in image 895
Multiple detections in image 912
No keypoints detected in image 958
No keypoints detected in image 965
No keypoints detected in image 992
Multiple detections in image 1140
No keypoints detected in image 1156
No keypoints detected in image 1234
No keypoints detected in image 1265
No keypoints detected in image 1316
No keypoints detected in image 1347
Multiple detections in image 1390
Multiple detections in image 1409
No keypo